# Черновик

In [77]:
try:
    conn = psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432
        
    )

except:
    print('Can`t establish connection to database')



In [ ]:

cur = conn.cursor()
with open('clients.csv', 'r') as f:
    next(f)  # Skip the header row.
    cur.copy_from(f, 'user3.clients', sep=',')
conn.commit()
cur.close()
conn.close()

In [ ]:
with open('clients.csv', 'r') as f:
    next(f)  # Skip the header row.
    with conn.cursor() as curs:
        cur.copy_from(f, 'ashkinadzi.clients', sep=',')
    

In [ ]:
with open('clients.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row['client_id'])

In [ ]:
with open('clients.csv', 'r') as f:
    next(f)
    print(f.readlines()[1:])

In [ ]:
list_of_csvs = ['clients.csv', 'securities.csv', 'security_transactions.csv', 'bank_transactions.csv']
list_of_tables = ['user3.clients', 'user3.securities', 'user3.security_transactions', 'user3.transactions']

with conn.cursor() as cur:

    try:
        for file, table in zip(list_of_csvs, list_of_tables):
            with open(file, 'r') as f:
                next(f)
                cur.copy_from(f, table, sep=',')
            #conn.commit()
            #cur.close()
            #conn.close()
    finally:
        #cur.close()
        #conn.close()
        print('OK')

In [ ]:
#Проверка соединения с базой

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    
    with conn.cursor() as curs:
        curs.execute('SELECT * FROM ashkinadzi.transactions_buff WHERE client_id = 3')
        all_clients = curs.fetchall()
        print(all_clients)

# Отсюда пошел чистовой код

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import psycopg2
import time
import csv
import requests
import xml.etree.ElementTree as ET


## Получение и преобразование в csv данных о валютах по API

In [19]:
def fetch_xml_from_cbr(url):
    """
    Загружает XML файл с сайта ЦБ РФ.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Проверяем статус ответа
        return response.text  # Возвращаем XML в виде строки
    except Exception as e:
        print(f"Ошибка при загрузке XML файла: {e}")
        return None

def parse_cbr_xml_to_dataframe(xml_data):
    """
    Преобразует XML данные из структуры ЦБ РФ в pandas DataFrame.
    """
    try:
        root = ET.fromstring(xml_data)  # Парсим XML
        rows = []

        # Ищем элементы <Valute>
        for valute in root.findall("Valute"):
            row = {
                "ID": valute.get("ID"),  # Атрибут ID
                "NumCode": valute.findtext("NumCode"),
                "CharCode": valute.findtext("CharCode"),
                "Nominal": int(valute.findtext("Nominal")),
                "Name": valute.findtext("Name"),
                "Value": float(valute.findtext("Value").replace(",", ".")),
                "Date": root.get("Date"),  # Дата из корневого элемента
            }
            rows.append(row)

        # Преобразуем список словарей в DataFrame
        df = pd.DataFrame(rows)
        return df
    except Exception as e:
        print(f"Ошибка при обработке XML: {e}")
        return pd.DataFrame()  # Возвращаем пустой DataFrame в случае ошибки

In [20]:
date = '11/12/2024'

if __name__ == "__main__":
    # Ссылка на XML файл ЦБ РФ
    url = f"https://www.cbr.ru/scripts/XML_daily.asp?date_req={date}"

    # Загрузка XML данных
    xml_data = fetch_xml_from_cbr(url)
    
    if xml_data:
        # Преобразуем XML в DataFrame
        df = parse_cbr_xml_to_dataframe(xml_data)
        df.set_index('ID', inplace=True)
        display(df.head())
        #Преобразование датафрейма в csv файл
        df.to_csv('currencies.csv', sep=";", encoding='utf8')



,NumCode,CharCode,Nominal,Name,Value,Date
ID,,,,,,
R01010,036,AUD,1,Австралийский доллар,63.8707,11.12.2024
R01020A,944,AZN,1,Азербайджанский манат,58.8426,11.12.2024
R01035,826,GBP,1,Фунт стерлингов Соединенного королевства,128.0015,11.12.2024
R01060,051,AMD,100,Армянских драмов,25.3330,11.12.2024
R01090B,933,BYN,1,Белорусский рубль,29.3790,11.12.2024


## Изменение разделителя таблиц clients и securities для корректной загрузки в postgres

In [22]:
#Изменение разделителя в исходном csv файле с запятой на точку с запятой для правильного парсинга столбцов для таблицы clients_buff

df = pd.read_csv('clients.csv', index_col='client_id')
df.head()
df.to_csv('clients2.csv', sep=";")

In [24]:
#Изменение разделителя в исходном csv файле с запятой на точку с запятой для правильного парсинга столбцов для таблицы securities_buff

df = pd.read_csv('securities.csv', index_col='security_id')
df.head()
df.to_csv('securities2.csv', sep=";")

## Загрузка данных в буферные таблицы

In [73]:
# Очистка буферных таблиц

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        cur.execute('TRUNCATE ashkinadzi.clients_buff, ashkinadzi.securities_buff, ashkinadzi.currencies_buff, ashkinadzi.security_transactions_buff, ashkinadzi.transactions_buff RESTART IDENTITY CASCADE')

In [74]:
#Заполнение таблицы clients_buff

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    with conn.cursor() as cur, open('clients2.csv', 'r') as f:
        cur.execute('SET search_path TO ashkinadzi')
        next(f)
        cur.copy_from(f, 'clients_buff', columns=['client_id', 'client_name', 'client_email', 'client_phone', 'client_address'], sep=";")

In [75]:
#Заполнение таблицы securities_buff

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    
    with conn.cursor() as cur, open('securities2.csv', 'r') as f:
        cur.execute('SET search_path TO ashkinadzi')
        next(f)
        cur.copy_from(f, 'securities_buff', columns=['security_id', 'security_name', 'security_type', 'market'], sep=";")

In [76]:
#Заполнение таблицы currencies_buff

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    
    with conn.cursor() as cur, open('currencies.csv', 'r', encoding="utf-8") as f:
        cur.execute('SET search_path TO ashkinadzi')
        next(f)
        cur.copy_from(f, 'currencies_buff', columns=['id', 'numcode', 'charcode', 'nominal', "name", 'value', "date"], sep=";")

In [77]:
#Заполнение таблицы security_transactions_buff

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    
    with conn.cursor() as cur, open('security_transactions.csv', 'r') as f:
        cur.execute('SET search_path TO ashkinadzi')
        next(f)
        cur.copy_from(f, 'security_transactions_buff', columns=['client_id', 'security_id', 'transaction_date', 'transaction_type', 'quantity', 'currency', 'price'], sep=",")

In [78]:
#Заполнение таблицы transactions_buff

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    
    with conn.cursor() as cur, open('bank_transactions.csv', 'r') as f:
        cur.execute('SET search_path TO ashkinadzi')
        next(f)
        cur.copy_from(f, 'transactions_buff', columns=['client_id', 'transaction_id', 'transaction_date', 'transaction_type', 'account_number', 'currency', 'amount'], sep=",")

## Перенос данных из буферных таблиц в базовые

In [79]:
# Вызов процедуры по переносу данных из буферной таблицы clients_buff в базовую clients_base

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        cur.execute('CALL buff_to_base_transfer_clients()')

In [80]:
# Вызов процедуры по переносу данных из буферной таблицы securities_buff в базовую securities_base

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        cur.execute('CALL buff_to_base_transfer_securities()')

In [81]:
# Вызов процедуры по переносу данных из буферной таблицы currencies_buff в базовую currencies_base

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        cur.execute('CALL buff_to_base_transfer_currencies()')

In [82]:
# Вызов процедуры по переносу данных из буферной таблицы security_transactions_buff в базовую security_transactions_base

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        cur.execute('CALL buff_to_base_transfer_security_transactions()')

In [83]:
# Вызов процедуры по переносу данных из буферной таблицы transactions_buff в базовую transactions_base

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        cur.execute('CALL buff_to_base_transfer_transactions()')

## Вспомогательные скрипты

In [72]:
# Очистка базовых таблиц (на всякий случай)

with psycopg2.connect(
        host='172.17.1.32',
        user='wave15_user_a3',
        password='pass',
        dbname='wave15_team_a',
        port=15432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        cur.execute('TRUNCATE ashkinadzi.clients_base, ashkinadzi.securities_base, ashkinadzi.currencies_base, ashkinadzi.security_transactions_base, ashkinadzi.transactions_base RESTART IDENTITY CASCADE')